In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

...

ImportError: numpy.core.umath failed to import

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [5]:
! pip install tensorflow

  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.33.0
    Uninstalling google-auth-1.33.0:
      Successfully uninstalled google-auth-1.33.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.22.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.22.0 which is incompatible.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.22.0 which is incompatible.


In [11]:
! pip install numpy==1.21.6

  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.21.6 which is incompatible.


In [8]:
# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [9]:
train = pd.read_csv('./train.csv')
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [7]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)   
    
    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    ####### train / test 분리


    train_size = int(len(data_y) * 0.7)
    train_X = np.array(data_X[0 : train_size])
    train_y = np.array(data_y[0 : train_size])

    test_size = len(data_y) - train_size
    test_X = np.array(data_X[train_size : len(data_X)])
    test_y = np.array(data_y[train_size : len(data_y)])

    print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
    print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

    ## 표준화 + 모델 진행
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'][train_close['종목코드']==code] = train_scaled

    # 변수의 가중치 화
    tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    final_return = np.mean(predictions)
    
    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

NameError: name 'train' is not defined

In [10]:
#### 0727 ver --- LSTM 이걸로 진행
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()
    
    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    ## 표준화 + 모델 진행
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'] = train_scaled

    # 데이터 전처리
    # 변수 x
    dfx = train_close[['5_10평균']]
    # 정답 y
    dfy = train_close[['종가']]

    X = dfx.values.tolist()
    y = dfy.values.tolist()

    ###  10일 동안의 OHLVC 데이터로 다음 날의 종가를 예측
    
    window_size = 15

    data_X = []
    data_y = []
    for i in range(len(y) - window_size):
        _X = X[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
        _y = y[i + window_size]     # 다음 날 종가
        data_X.append(_X)
        data_y.append(_y)
    print(_X, "->", _y)


    ####### train / test 분리

    print('전체 데이터의 크기 :', len(data_X), len(data_y))
    
    train_size = int(len(data_y) * 0.8)
    train_X = np.array(data_X[0 : train_size])
    train_y = np.array(data_y[0 : train_size])

    test_size = len(data_y) - train_size
    test_X = np.array(data_X[train_size : len(data_X)])
    test_y = np.array(data_y[train_size : len(data_y)])

    print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
    print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

    # 모델 학습
    model = Sequential()
    model.add(LSTM(units=20, activation='relu', return_sequences=True, input_shape=(15, 1)))
    model.add(Dropout(0.1))
    model.add(LSTM(units=20, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.summary()

    # 모델 학습
    predictions =[]
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(train_X, train_y, epochs=70, batch_size=30)
    
    predictions.append(model.predict(test_X)) # 예측
    
    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': predictions}, ignore_index=True)


final_df


  0%|          | 0/2000 [00:00<?, ?it/s]

[[-0.08188499423395078], [0.33765899533891747], [0.4683366314353846], [0.5852587268901184], [0.7090585926657189], [0.6334031191361853], [0.3789256172641176], [0.2963923734137173], [0.22073689988418368], [0.09693703410858323], [0.15883696699638347], [0.2963923734137173], [0.37204784694325094], [0.3170256843763174], [1.156113663522054]] -> [3015]
전체 데이터의 크기 : 420 420
훈련 데이터의 크기 : (336, 15, 1) (336, 1)
테스트 데이터의 크기 : (84, 15, 1) (84, 1)


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [4]:
#### 0725 ver --- 이거로 진행
# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()
date = train['일자'].unique()

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드'])
final_df = final_df.set_index(date)


for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)   
    
    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    #train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    # 학습 데이터프레임
    for i in range(len(train_close)):
        train_close['5_10평균'][i]


    final_df = final_df.append({'종목코드': code, 'final_return': train_close['5_10평균']}, ignore_index=True)
    final_df = final_df.merge([final_df,],on=final_df.index)

final_df

100%|██████████| 2000/2000 [01:59<00:00, 16.69it/s]


,종목코드,final_return
0,A060310,일자 2021-08-24 -144.5 2021-08-25 -120.5 202...
1,A095570,일자 2021-08-24 -101.0 2021-08-25 -121.0 202...
2,A006840,일자 2021-08-24 -1095.0 2021-08-25 -920.0 2...
3,A054620,일자 2021-08-24 -1065.0 2021-08-25 -1235.0 2...
4,A265520,일자 2021-08-24 -1020.0 2021-08-25 -765.0 2...
...,...,...
1995,A189980,일자 2021-08-24 -264.0 2021-08-25 -262.0 202...
1996,A000540,일자 2021-08-24 -94.0 2021-08-25 -74.5 2021-...
1997,A003280,일자 2021-08-24 0.0 2021-08-25 0.0 2021-...
1998,A037440,일자 2021-08-24 -219.0 2021-08-25 -203.0 202...


In [15]:
final_df.iloc[0,1]

일자
2021-08-24   -144.5
2021-08-25   -120.5
2021-08-26    -76.5
2021-08-27      8.5
2021-08-30     63.0
              ...  
2023-05-23     15.5
2023-05-24     21.0
2023-05-25     17.0
2023-05-26     78.0
2023-05-30    140.5
Name: 5_10평균, Length: 435, dtype: float64

In [ ]:
https://wikidocs.net/173005